<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/3_make_errors/e6_make_errors_modify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kiwipiepy
# !pip install nltk
# !pip install
# !pip install selenium

#코랩에서 구글드라이버 연동
# !apt-get update
# !apt install chromium-chromedriver

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

Mounted at /content/drive


# 0.Data Load

In [1]:
import pandas as pd
import numpy as np
import json
import os
import re
import copy
import pickle


if os.path.exists(r"G:\내 드라이브\KB인턴관련"):
    gdrive_path = r"G:\내 드라이브\KB인턴관련"
else:
    gdrive_path = "/content/drive/MyDrive/KB인턴관련"

In [2]:
df = pd.read_csv(os.path.join(gdrive_path, "preprocessing/fin.csv"), encoding="euc-kr", index_col=0)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280765 entries, 0 to 585940
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   category  280765 non-null  object
 1   lawName   280765 non-null  object
 2   ENG       280765 non-null  object
 3   KOR       280765 non-null  object
dtypes: object(4)
memory usage: 10.7+ MB


# 1.Data Fiiltering

In [4]:
keyword_idx = df.loc[df.lawName.str.contains('은행|금융|보험|예금|적금|대출|외환|통화|화폐|환율|예대')|df.category.str.contains('경제|금융|상업')].index

In [5]:
tgk_idx = df.loc[df.category=="통화ㆍ국채ㆍ금융"].index

In [6]:
print(f"키워드: {len(keyword_idx)}\t통화/국채/금융: {len(tgk_idx)}")
print(f"키워드-통국금: {len(list(set(keyword_idx)-set(tgk_idx)))}")
print(f"통국금-키워드: {len(list(set(tgk_idx)-set(keyword_idx)))}")
print(f"전체:  {len(list(set(keyword_idx).union(set(tgk_idx))))}")

키워드: 45093	통화/국채/금융: 16004
키워드-통국금: 29089
통국금-키워드: 0
전체:  45093


In [7]:
df_samp = df.loc[list(set(keyword_idx).union(set(tgk_idx)))].drop(index=[403163], axis=0)

In [8]:
nfin_idx = list(set(df.index)-set(df_samp.index))
fin_idx = list(df_samp.index)
print(len(nfin_idx), len(fin_idx))

235673 45092


**주요 함수**

In [9]:
dindex_dict = {"omission":[], "addition":[], "mis-translation":[], "over-translation":[], "under-translation":[], "grammer": []}

In [10]:
## 주요한 함수
def remake_df():
    result = df.loc[list(set(keyword_idx).union(set(tgk_idx)))]
    result["curHyp"] = result.KOR.values
    result["errors"] = [[] for _ in range(result.shape[0])]
    result["errLog"] = [dict() for _ in range(result.shape[0])]
    return result.drop(index=[403163], axis=0)

def random_df(dfs, ratio, seed):
    np.random.seed(seed)
    return dfs.loc[np.random.choice(dfs.index.to_list(), int(dfs.shape[0]*ratio), False), :]

def random_index(srs, ratio, seed):
    np.random.seed(seed)
    return np.random.choice(srs.index, int(srs.shape[0]*ratio), False)

def copy_df(data, col=["errors", "errLog"]):
    result = data.drop(columns=col, axis=1).copy(deep=True)
    # print(id(result), id(data))
    for c in col:
        result[c] = data[c].apply(lambda x: copy.deepcopy(x)).copy(deep=True)
    return result

def count_tag(tokens, tags=None):
    token_dict = dict()
    for token in tokens:
        if tags!=None and token.tag not in tags: continue
        try:
            token_dict[token.tag].append(token.form)
        except:
            token_dict[token.tag] = [token.form]

    return token_dict

def find_tokens(tokens, tag):
    result_tokens = []
    for token in tokens:
        if token.tag == tag:
            result_tokens.append((token.form, token.start, token.end))
    return result_tokens

def print_df(df, cols, idx=0, end=0):
    print(df.shape[0])
    if end==0:
        values = df.index[idx:]
    else:
        values = df.index[idx:end]
    for i in values:
        print(f"_________________ {i} _________________")
        for col in cols:
            print(df.loc[i, col])

def input_errors(df, index, severity, start_idx, text, type):
    df.loc[index, "errors"].append({"severity":severity, "start_idx":start_idx, "text":text, "type":type})

def convert_df(df, diff_df):
    cdf = copy_df(df)
    for i in diff_df.index:
        cdf.loc[i, : ] = diff_df.loc[i, : ]
    return cdf

def dup_drop():
    for k in dindex_dict.keys():
        dindex_dict[k] = list(set(dindex_dict[k]))

def check_idx(df):
    dup_drop()
    for k in dindex_dict.keys():
        print(f"\t  {k}: {len(dindex_dict[k])}", end='\t')
        print(df.loc[df.errLog.apply(lambda x: k in x.keys())].shape[0])

def check_memorize(data):
    df = copy_df(data.loc[:, ["curHyp","errLog"]], col=["errLog"])
    result_list = []
    for i in df.index:
        cur_hyp = df.loc[i, "curHyp"]
        d = df.loc[i, "errLog"]
        result_dict = dict()
        for k in d.keys():
            start = d[k]["start_idx"]
            end = start+len(d[k]["text"])
            result_dict[k] = cur_hyp[start:end]
        result_list.append(result_dict)
    df["validation"] = result_list
    return df

def move_index(df, idx, start, word):
    err_log = copy.deepcopy(df.loc[idx, "errLog"])
    loop_keys = err_log.keys()
    for k in loop_keys:
        pre_idx = err_log[k]["start_idx"]
        if pre_idx > start:
            df.loc[idx, "errLog"][k]["start_idx"] = pre_idx + (len(word)+1)

def replace_inteli(df, idx, keyword, rword):
    errLog = df.loc[idx ,"errLog"]
    cur_hyp = df.loc[idx, "curHyp"]

    text_dict=dict()
    for k in errLog.keys():
        if k=="omission": continue
        text, sindex = errLog[k]["text"], errLog[k]["start_idx"]
        text_dict[text] = (sindex, sindex+len(text))

    reject_range = []
    for key in text_dict.keys():
        if keyword in key:
            reject_range.append(text_dict[key])
    try:
        for f in re.finditer(keyword, cur_hyp):
            start, end = f.start(), f.end()
            is_reject = False
            for r_s, r_e in reject_range:
                if r_s <= start < r_e:
                    is_reject=True
                elif start <= r_s < r_e <= end:
                    is_rejcet=True

            if is_reject: continue
            while start>=0 and cur_hyp[start]!=" ":
                start-=1
            while end<len(cur_hyp) and cur_hyp[end]!=" ":
                end+=1
            start += 1

            word = cur_hyp[start:end]

            if word in text_dict.keys():
                start_idx = text_dict[word]
                if start==start_idx: continue
                break
            break
    except:
        raise print(f"{keyword} {rword}\n{cur_hyp}")

    try:
        add_word = word.replace(keyword, rword)
        conv_sentence = cur_hyp[:start]+add_word+cur_hyp[start+len(word):]
        return conv_sentence, start, add_word
    except:
        return None


def can_noise(df, i):
    cur_hyp = df.loc[i, "curHyp"]
    errLog = df.loc[i, "errLog"]
    for k in errLog.keys():
        if k=="omission": continue
        start, text = errLog[k]["start_idx"], errLog[k]["text"]
        end = start+len(text)
        cur_hyp = cur_hyp[:start]+("ㅱ"*len(text))+cur_hyp[end:]
    return cur_hyp

def errors_report(df, rate=False):
    sevs = {"critical":0, "major":1, "minor":2, "neutral":3}
    err_types = {
        'omission':[0, 0, 0, 0],
        'addition':[0, 0, 0, 0],
        'mis-translation':[0, 0, 0, 0],
        'over-translation':[0, 0, 0, 0],
        'under-translation':[0, 0, 0, 0],
        'grammer':[0, 0, 0, 0]
        }
    for i in df.index:
        errors = df.loc[i, "errors"]
        for d in errors:
            err_types[d["type"]][sevs[d["severity"]]]+=1
    report_df = pd.DataFrame(err_types, index=sevs.keys())
    report_df["sum"] = report_df.apply(np.sum, axis=1)
    report_df.loc["sum"] = report_df.apply(np.sum, axis=0)
    if rate:
        return_df = pd.DataFrame()
        for col in report_df.columns:
            return_df[col] = report_df[col].apply(lambda x: round((100*x)/report_df.loc["sum", "sum"], 2))
        return return_df
    else:
        return report_df

In [11]:
df_nfin = random_df(df.loc[nfin_idx], 0.3, 111)
df_fin = random_df(df.loc[fin_idx], 0.8, 123)
df_samp = pd.concat([df_nfin, df_fin], axis=0)
df_samp["curHyp"] = df_samp.KOR.values
df_samp["errors"] = [[] for _ in range(df_samp.shape[0])]
df_samp["errLog"] = [dict() for _ in range(df_samp.shape[0])]

# 2.품사 태깅

In [12]:
# from kiwipiepy import Kiwi

# kiwi = Kiwi(num_workers=0, model_type='sbg')
# kiwi_tokens = df_samp.KOR.apply(lambda x: kiwi.tokenize(x))
# count_tokens = kiwi_tokens.apply(count_tag)

In [13]:
# nng_dict = dict()
# for tokens in count_tokens.apply(lambda x: x["NNG"]):
#     for t in tokens:
#         try:
#             nng_dict[t] += 1
#         except:
#             nng_dict[t] = 1
# tag = "NNG"
# nouns =list(set(count_tokens.apply(lambda x: x[tag] if tag in x.keys() else []).sum()))

# 3.Error 생성

In [14]:
dindex_dict = {"omission":[], "addition":[], "mis-translation":[], "over-translation":[], "under-translation":[], "grammer": []}

## 3-1.Error 발생 함수

In [53]:
# -- Omission : Critical
def critical_omission(df, ratio, seed):
    df_t = copy_df(df)
    cr_om_index = random_index(df_t.curHyp, ratio, seed)
    dindex_dict["omission"]+=list(cr_om_index)
    result = df_t.loc[cr_om_index]
    random_ratio = [3/5, 2/3, 5/8, 5/7]
    np.random.seed(seed)
    split_sentences = result.curHyp.apply(lambda x: x.split(" "))
    len_element = split_sentences.apply(len)
    crop_element = len_element.apply(lambda x: int(x*(np.random.choice(random_ratio, 1))))
    start_idx = (len_element-crop_element).apply(lambda x: np.random.choice([i for i in range(x)], 1)[0])
    end_idx = start_idx+crop_element

    for i in cr_om_index:
        reduce_sentence = (" ".join([split_sentences[i][k] for k in range(start_idx[i], end_idx[i]+1)])).strip()
        cur_hyp = result.loc[i, "curHyp"]
        sidx = cur_hyp.find(reduce_sentence)

        conv_hyp = " ".join([split_sentences[i][j] for j in range(len_element[i]) if (start_idx[i] > j) or (end_idx[i] < j)])
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "critical", sidx, reduce_sentence, "omission")
        result.loc[i, "errLog"]["omission"] = {"start_idx": sidx, "text": reduce_sentence}

    del split_sentences, len_element, crop_element, start_idx, end_idx
    return convert_df(df, result)

# -- Omission : Major
def major_omission(df_local, ratio, seed, tag, slen, kiwi_tokens):
    df_t = copy_df(df_local)
    np.random.seed(seed)
    nnp_info = kiwi_tokens.apply(lambda x: find_tokens(x, tag)).drop(index=dindex_dict["omission"], axis=0)
    np.random.seed(seed)
    change_val = nnp_info[nnp_info.apply(len) > 0].apply(lambda x: x[np.random.choice(len(x), 1)[0]])
    ch6 = change_val[change_val.apply(lambda x: len(x[0])) > slen]
    chv_index = random_index(ch6, ratio, seed)
    # drop_idx = ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1].index.to_list()
    ch_srs = ch6.loc[chv_index]
    result = df_t.loc[chv_index]

    for i in chv_index:
        del_word, start, end = ch_srs.loc[i]
        cur_hyp = result.loc[i, "curHyp"]

        forward = cur_hyp[:start]
        backward = cur_hyp[start:]
        split_back = backward.split(" ")
        del_unit = split_back[0]
        back_mod = " ".join(split_back[1:])

        try:
            if start==0:
                conv_hyp = (forward+back_mod).strip(" ")
            elif cur_hyp[start-1] == '"' or cur_hyp[start-1]=='「' or cur_hyp[end]=="」" or cur_hyp[end]=='\"' :
                pattern = f'\"[^\"]*{del_word}[^\"]*\"|「[^」]*{del_word}[^」]*」'
                find_comp = re.compile(pattern)
                find_res = find_comp.search(cur_hyp)
                start, end = find_res.span()
                while cur_hyp[end]!=" " and  end<len(cur_hyp):
                    end+=1
                while cur_hyp[start]!=" " and start>=0:
                    start-=1
                start+=1
                del_unit = cur_hyp[start:end]
                conv_hyp = (cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()).strip()
            elif "(" in del_unit:
                pattern = f'{del_word}[^\)]+\)'
                find_comp = re.compile(pattern)
                find_res = find_comp.search(cur_hyp)
                start, end = find_res.span()
                while cur_hyp[end]!=" " and  end<len(cur_hyp):
                    end+=1
                while cur_hyp[start]!=" " and start>=0:
                    start-=1
                start+=1
                del_unit = cur_hyp[start:end]
                conv_hyp = (cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()).strip()
            else:
                conv_hyp = (forward+back_mod).strip(" ")
            dindex_dict["omission"]+=[i]
            result.loc[i, "curHyp"] = conv_hyp
            input_errors(result, i, "major", start, del_unit, "omission")
            result.loc[i, "errLog"]["omission"] = {"start_idx": start, "text": del_unit}
        except:
            pass

    return convert_df(df_local, result)
## -- Omission : Neutral
def omission_nt(df, ratio, seed):    # 0.01, 1, 11
    df_t = copy_df(df)
    np.random.seed(seed)
    on_index = random_index(df_t.drop(index=dindex_dict["omission"]), ratio, seed)
    on_index = df_t.loc[on_index][df_t.loc[on_index].curHyp.str.contains(r"\.|\;|\:")].index
    result = df_t.loc[on_index]
    dindex_dict["omission"]+=list(on_index)

    sp_list = [".", ";", ":"]
    for i in on_index:
        np.random.shuffle(sp_list)
        for s in sp_list:
            cur_hyp = result.loc[i, "curHyp"]
            s_loc = cur_hyp.find(s)
            if s_loc==-1:
                continue
            conv_hyp = cur_hyp[:s_loc]+cur_hyp[s_loc+1:]
            result.loc[i, "curHyp"] = conv_hyp
            input_errors(result, i, "neutral", s_loc, s, "omission")
            result.loc[i, "errLog"]["omission"] = {"start_idx": s_loc, "text": s}
            break
    return convert_df(df, result)

## Addition : Major
def major_addition(df, ratio, seed, count_tokens, nouns):
    np.random.seed(seed)
    df_t=copy_df(df)

    ma_ad_index = random_index(df_t, ratio, seed)
    nouns_lists = count_tokens[ma_ad_index].apply(lambda x: x["NNG"])
    nnpg_info = count_tokens[ma_ad_index].apply(lambda x: [val for val in list(set(x["NNG"])) if len(val)>1])
    drop_index = nnpg_info[nnpg_info.apply(len) == 0].index.to_list()
    nnpg_info.drop(index=drop_index, axis=0, inplace=True)
    ma_ad_index = list(set(ma_ad_index)-set(drop_index))
    np.random.seed(seed)
    add_position = nnpg_info.apply(lambda x: x[np.random.choice(range(0, len(x)))])
    np.random.seed(seed)
    add_words = nouns_lists.apply(lambda x: np.random.choice(list(set(nouns)-set(x))))
    result = df_t.loc[ma_ad_index]


    josa_list = ["의", "에 의한", "에", "만", "만의"]
    for i in ma_ad_index:
        cur_hyp = df.loc[i, "curHyp"]
        josa_add = np.random.choice(josa_list)
        word = add_position[i]
        start = cur_hyp.find(word)
        if word not in cur_hyp:
            continue
        add_word = add_words[i]+josa_add
        while start>=0 and cur_hyp[start]!=" ":
            start-=1
        if start < 0:
            conv_hyp = f"{add_word}"+" "+cur_hyp[start+1:].strip()
        else:
            conv_hyp = cur_hyp[:start+1].strip()+" "+f"{add_word}"+" "+cur_hyp[start+1:].strip()
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "major", start+1, add_word, "addition")

        move_index(result, i, start+1, add_word)
        result.loc[i, "errLog"]["addition"] = {"start_idx": (start+1), "text": add_word}
        dindex_dict["addition"].append(i)

    return convert_df(df, result)

## -- Addition : Minor
def ma_iter(df, ratio, seed):   #0.2 2, 34
    df_t = copy_df(df)
    np.random.seed(seed)
    am_index = random_index(df_t.drop(index=dindex_dict["addition"]), ratio, seed)
    result = df_t.loc[am_index]
    dindex_dict["addition"]+=list(am_index)

    for i in am_index:
        cur_hyp = result.loc[i, "curHyp"]
        sch = cur_hyp.split(" ")
        rand_idx = np.random.choice([i for i in range(len(sch))])
        random_dup = sch[rand_idx]
        conv_hyp=" ".join(sch[:rand_idx] + [random_dup] +sch[rand_idx:])
        if rand_idx==0:
            start = 0
        else:
            start = len(" ".join(sch[:rand_idx]))+1

        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "minor", start, random_dup, "addition")

        move_index(result, i, start, random_dup)
        result.loc[i, "errLog"]["addition"] = {"start_idx": start, "text": random_dup}

    return convert_df(df, result)

## Mis-Translation : Minor
def hv_mistrans(df, ratio, seed):   # 0.3 7, 89
    df_t = copy_df(df)
    te_drop = df_t.drop(index=dindex_dict["mis-translation"])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    temp = te_drop.loc[transHyp.str.contains(r'하여야 한다')]
    can_index = random_index(temp, ratio, seed)
    result = te_drop.loc[can_index]
    ch_words = ['해야만 한다', '해야 할 것이다']
    dindex_dict["mis-translation"] += list(can_index)

    for i in can_index:
        np.random.shuffle(ch_words)
        ch_word = ch_words[0]
        cur_hyp = result.loc[i, "curHyp"]
        start = cur_hyp.find('하여야 한다')
        conv_hyp = cur_hyp[:start] + ch_word + cur_hyp[start+6:]

        result.loc[i, 'curHyp'] = conv_hyp
        input_errors(result, i, 'minor', start, ch_word, 'mis-translation')

        move_index(result, i, start, ch_word)
        result.loc[i, "errLog"]["mis-translation"] = {"start_idx": start, "text": ch_word}
    return convert_df(df, result)

def ddo_trans(df, ratio, seed, val):
    df_t = copy_df(df)
    conv_list = ["내지", "및", "또는"]
    conv_list.remove(val)
    te_drop = df_t.drop(index=dindex_dict["mis-translation"])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    ival = " "+val+" "
    ddo_idx = random_index(te_drop.loc[transHyp.str.contains(ival)], ratio, seed)
    result = te_drop.loc[ddo_idx]
    dindex_dict["mis-translation"]+=list(ddo_idx)


    for i in ddo_idx:
        np.random.shuffle(conv_list)
        conv_word = conv_list[0]

        conv_hyp, start, fconv_word = replace_inteli(result, i, val, conv_word)

        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, 'minor', start, fconv_word, 'mis-translation')

        move_index(result, i, start, fconv_word)
        result.loc[i, "errLog"]["mis-translation"] = {"start_idx": start, "text": fconv_word}
    return convert_df(df, result)

## -- Mis-Translation : Major
def replace_numbers(df, ratio, seed):
    df_t = copy_df(df)
    patterns = r"제[\d]+[\w]+[\d]+[\w]+\s"
    compiler = re.compile(patterns)
    te_drop = df_t.drop(index=dindex_dict["mis-translation"])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    std_idx = random_index(te_drop.loc[transHyp.str.contains(patterns)], ratio, seed)
    result = te_drop.loc[std_idx]

    for i in std_idx:
        cur_hyp = result.loc[i, "curHyp"]
        all_find = list(compiler.finditer(cur_hyp))
        np.random.shuffle(all_find)
        for f in all_find:
            sts = f.group().strip()
            sts_nsp = re.sub(r"[가-힣]+", " ", sts)
            sts_list = re.split(r"[^\d]+", sts_nsp)[1:-1]
            sts_add = re.split(r"[\d]+", sts)
            pre_shuffle = copy.deepcopy(sts_list)
            if len(list(set(pre_shuffle)))==1: continue
            while True:
                np.random.shuffle(pre_shuffle)
                if sts_list!=pre_shuffle:
                    break
        chyp = sts_add.pop(0)
        for a, b in zip(sts_add, pre_shuffle):
            chyp+=(b+a)

        conv_hyp, start, conv_word = replace_inteli(result, i, sts, chyp)

        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, 'major', start, conv_word, 'mis-translation')

        move_index(result, i, start, conv_word)
        result.loc[i, "errLog"]["mis-translation"] = {"start_idx": start, "text": conv_word}
        dindex_dict["mis-translation"].append(i)
    return convert_df(df, result)

## -- Mis-Translation : Neutral
def change_morphus(df, ratio, seed, rep_dict):
    df_t = copy_df(df)
    patterns = "|".join(rep_dict.keys())

    te_drop = df_t.drop(index=dindex_dict["mis-translation"])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    std_idx = random_index(te_drop.loc[transHyp.str.contains(patterns, regex=True)], ratio, seed)

    result = te_drop.loc[std_idx]

    rpd_keys = copy.deepcopy(list(rep_dict.keys()))
    for e, i in enumerate(std_idx):
        cur_hyp = result.loc[i, "curHyp"]
        np.random.shuffle(rpd_keys)
        for pattern in rpd_keys:
            find_list = list(re.finditer(pattern, cur_hyp))
            if not find_list: continue
            break

        np.random.shuffle(find_list)
        f = find_list[0]

        word, start = f.group(), f.start()
        r_word = (rep_dict[pattern]+word[1:-1]+rep_dict[pattern]).strip()
        res = replace_inteli(result, i, word, r_word)
        if res!=None:
            conv_hyp, start, conv_word = res
        else:
            continue

        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, 'neutral', start, conv_word, 'mis-translation')

        move_index(result, i, start, conv_word)
        result.loc[i, "errLog"]["mis-translation"] = {"start_idx": start, "text": conv_word}
        dindex_dict["mis-translation"].append(i)
    return convert_df(df, result)

def change_word(df, ratio, seed, rep_dict, severity, errType):
    df_t = copy_df(df)
    patterns = "|".join(rep_dict.keys())

    te_drop = df_t.drop(index=dindex_dict[errType])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    std_idx = random_index(te_drop.loc[transHyp.str.contains(patterns, regex=True)], ratio, seed)

    result = te_drop.loc[std_idx]

    rpd_keys = copy.deepcopy(list(rep_dict.keys()))
    for e, i in enumerate(std_idx):
        cur_hyp = result.loc[i, "curHyp"]
        np.random.shuffle(rpd_keys)
        is_complete=False
        for pattern in rpd_keys:
            i_word = pattern
            o_word = rep_dict[pattern][np.random.choice(list(range(0, len(rep_dict[pattern]))))]

            find_list = list(re.finditer(pattern, cur_hyp))
            if len(find_list)==0: continue
            break
        np.random.shuffle(find_list)
        f = find_list[0]
        start, init_word = f.start(), f.group()
        res = replace_inteli(result, i, i_word, o_word)
        if res != None:
            conv_hyp, start, conv_word = res
        else:
            continue
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, severity, start, conv_word, errType)

        move_index(result, i, start, conv_word)
        result.loc[i, "errLog"][errType] = {"start_idx": start, "text": conv_word}
        dindex_dict[errType].append(i)
    return convert_df(df, result)

## -- Grammer
def josa_grammer(df, ratio, seed, kiwi):
    df_t = copy_df(df)
    js_mstr_index = random_index(df_t.drop(index=dindex_dict["grammer"]), ratio, seed)
    js_tokens = df_t.loc[js_mstr_index].curHyp.apply(lambda x: kiwi.tokenize(x))

    jsd = {
        "가":"이", "을":"를", "로써":"으로써", "로서":"으로서",
        "로":"으로", "로부터":"으로부터", "란":"이란", "나":"이나",
        "와":"과", "랑":"이랑", "이랑":"랑", "며":"이며", "이며":"며"
        }
    keys = list(jsd.keys())
    for key in keys:
        jsd[jsd[key]] = key

    tags=["JKS", "JKC", "JKO", "JKB", "JX", "JC"]
    need_josa = None
    for i, tag in enumerate(tags):
        if i==0:
            need_josa = js_tokens.apply(lambda x: find_tokens(x, tag))
        else:
            need_josa += js_tokens.apply(lambda x: find_tokens(x, tag))

    drop_index = need_josa[need_josa.apply(len)==0].index
    js_mstr_index = list(set(js_mstr_index)-set(drop_index))
    need_josa.drop(index=drop_index, axis=0, inplace=True)
    josa_elem = need_josa.apply(lambda x: list(set([val[0] for val in x])))
    result = df_t.loc[js_mstr_index]

    for i in js_mstr_index:
        cur_hyp = result.loc[i, "curHyp"]
        target = josa_elem[i]
        np.random.shuffle(target)
        for word in target:
            try:
                conv_hyp, start, conv_word = replace_inteli(result, i, word, jsd[word])
                result.loc[i, "curHyp"] = conv_hyp
                input_errors(result, i, 'minor', start, conv_word, 'grammer')

                move_index(result, i, start, conv_word)
                result.loc[i, "errLog"]["grammer"] = {"start_idx": start, "text": conv_word}
                dindex_dict["grammer"].append(i)
                break
            except:
                pass

    return convert_df(df, result)

def hada_gram(df, ratio, seed):
    df_t = copy_df(df)
    te_drop = df_t.drop(index=dindex_dict["grammer"])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    temp = te_drop.loc[transHyp.apply(lambda x: "한다" in x)]
    handa_index = random_index(temp, 0.4, 141)
    result = df_t.loc[handa_index]
    replace_list = ["하다", "함"]

    for i in handa_index:
        dindex_dict["grammer"].append(i)
        cur_hyp = result.loc[i, "curHyp"]
        r_word = np.random.choice(replace_list)
        conv_hyp, start, conv_word = replace_inteli(result, i, "한다", r_word)

        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, 'minor', start, conv_word, 'grammer')

        move_index(result, i, start, conv_word)
        result.loc[i, "errLog"]["grammer"] = {"start_idx": start, "text": conv_word}

    return convert_df(df, result)

## -- 후처리
def convert_index(df_new):
    df = copy_df(df_new)
    new_errs = []
    for idx in df.index:
        errors, errLog = df.loc[idx, "errors"], df.loc[idx, "errLog"]

        for i, err in enumerate(df.loc[i, "errors"]):
            errors[i]['start_idx'] = errLog[err["type"]]["start_idx"]

        new_errs.append(errors)
    df["errors"] = new_errs
    return df

def make_dict(df):
    output_json = dict()
    output_json["data"] = []
    for i in df.index:
        base_form = {"category": None, "en": None, "errs": None, "hyp": None, "ko": None, "law_name": None}
        if len(df.loc[i, "errors"])==0:
            continue
        base_form["category"] = df.loc[i, "category"]
        base_form["en"] = df.loc[i, "ENG"]
        base_form["errs"] = df.loc[i, "errors"]
        base_form["hyp"] = df.loc[i, "curHyp"]
        base_form["ko"] = df.loc[i, "KOR"]
        base_form["law_name"] = df.loc[i, "lawName"]
        output_json["data"].append(base_form)
    return output_json

def export_json(objs, file_name):
    with open(os.path.join(gdrive_path, file_name), "w") as f:
        json.dump(objs, f, ensure_ascii=False, indent=4)

def step_index(df, errType, to_len=False):
    temp = check_memorize(df)
    ttp = temp.loc[temp.errLog.apply(lambda i: errType in i)]
    df_valid = ttp.loc[ttp["errLog"].apply(lambda x: x[errType]["text"]) != ttp["validation"].apply(lambda y: y[errType])]
    if to_len:
        return df_valid.shape[0]
    return df_valid

## 3-2.사전 정의 필요 변수

In [16]:
with open(os.path.join(gdrive_path, "etc_files/under_crawl.pkl"), "rb") as f:
    under = pickle.load(f)

with open(os.path.join(gdrive_path, "etc_files/over_crawl.pkl"), "rb") as f:
    over = pickle.load(f)

chmorp_dict = {
    r'\"[^\"]+\"':"'", r"\'[^\']+\'":'"',
    r'「[^」]+」':'"'
    }
chw_dict={r"\s때":[" 경우"], r"\s경우":[" 때"]}

In [17]:
ratios = [
    .02, .65, .55, .01, .22,
    .2, .3, .2, .25, .3,
    .35, .025, .3, .2, .2,
    .2 , .4
]

seed1 = [
    10, 20, 36, 111, 55,
    234, 789, 555, 444, 333,
    987, 777, 908, 808, 707,
    1010, 141
]

np.random.seed(111)
seed2 = np.random.choice(1000, 17, replace=False)
np.random.seed(123)
seed3 = np.random.choice(1000, 17, replace=False)
seed3

array([131, 203,  50, 585, 138, 632, 328, 461, 924, 479, 195, 221, 209,
       378, 156, 327, 761])

## 3-3.한번에 돌리는 함수

In [57]:
from kiwipiepy import Kiwi

def lets_make_errors(df, ratios, seeds):
    print("Tokenizing ...")
    kiwi = Kiwi(num_workers=0, model_type='sbg')
    kiwi_tokens = df.KOR.apply(lambda x: kiwi.tokenize(x))
    count_tokens = kiwi_tokens.apply(count_tag)

    nouns_set = set()
    for tokens in count_tokens:
        for key in tokens.keys():
            if key=="NNG":
                nouns_set = nouns_set.union(set(tokens[key]))
                break
    nouns = list(nouns_set)
    print("Complete Tokenizing !!!")

    print('\t1번째 수행 중...', end="")
    df_1 = critical_omission(df, ratios[0], seeds[0])
    print("완료 !")
    # if step_index(df_1, "omission", True) != 0:
    #   return df_5

    print('\t2번째 수행 중...', end="")
    df_2 = major_omission(df_1, ratios[1], seeds[1], "NNP", 3, kiwi_tokens)
    print("완료 !")
    # if  step_index(df_2, "omission", True).shape[0] != 0:
        # return df_5

    print('\t3번째 수행 중...', end="")
    df_3 = major_omission(df_2, ratios[2], seeds[2], "NNG", 2, kiwi_tokens)
    print("완료 !")
    # step_index(df1, "omission", True)

    print('\t4번째 수행 중...', end="")
    df_4 = omission_nt(df_3, ratios[3], seeds[3])
    print("완료 !")
    # step_index(df1, "omission", True)

    print('\t5번째 수행 중...', end="")
    df_5 = major_addition(df_4, ratios[4], seeds[4], count_tokens, nouns)
    print("완료 !")
    if step_index(df_5, "addition", True) != 0:
        return df_5

    print('\t6번째 수행 중...', end="")
    df_6 = ma_iter(df_5, ratios[5], seeds[5])
    print("완료 !")
    if step_index(df_6, "addition", True) != 0:
        return df_6

    print('\t7번째 수행 중...', end="")
    df_7 = hv_mistrans(df_6, ratios[6], seeds[6])
    print("완료 !")
    if step_index(df_7, "mis-translation", True) != 0:
        return df_7

    print('\t8번째 수행 중...', end="")
    df_8 = ddo_trans(df_7, ratios[7], seeds[7], "또는")
    print("완료 !")
    if step_index(df_8, "mis-translation", True) != 0:
        return df_8

    print('\t9번째 수행 중...', end="")
    df_9 = ddo_trans(df_8, ratios[8], seeds[8], "내지")
    print("완료 !")
    if step_index(df_9, "mis-translation", True) != 0:
        return df_9

    print('\t10번째 수행 중...', end="")
    df_10 = ddo_trans(df_9, ratios[9], seeds[9], "및")
    print("완료 !")
    if step_index(df_10, "mis-translation", True) != 0:
        return df_10

    print('\t11번째 수행 중...', end="")
    df_11 = replace_numbers(df_10, ratios[10], seeds[10])
    print("완료 !")
    if step_index(df_11, "mis-translation", True) != 0:
        return df_11

    print('\t12번째 수행 중...', end="")
    df_12 = change_morphus(df_11, ratios[11], seeds[11], chmorp_dict)
    print("완료 !")
    if step_index(df_12, "mis-translation", True) != 0:
        return df_12

    print('\t13번째 수행 중...', end="")
    df_13 = change_word(df_12, ratios[12], seeds[12], chw_dict, "minor", "mis-translation")
    print("완료 !")
    if step_index(df_13, "mis-translation", True) != 0:
        return df_13

    print('\t14번째 수행 중...', end="")
    df_14 = change_word(df_13, ratios[13], seeds[13], over, "major", "over-translation")
    print("완료 !")
    if step_index(df_14, "over-translation", True) != 0:
        return df_14

    print('\t15번째 수행 중...', end="")
    df_15 = change_word(df_14, ratios[14], seeds[14], under, "major", "under-translation")
    print("완료 !")
    if step_index(df_15, "under-translation", True) != 0:
        return df_15

    print('\t16번째 수행 중...', end="")
    df_16 = josa_grammer(df_15, ratios[15], seeds[15], kiwi)
    print("완료 !")
    if step_index(df_16, "grammer", True) != 0:
        return df_16

    print('\t17번째 수행 중...', end="")
    df_17 = hada_gram(df_16, ratios[16], seeds[16])
    print("완료 !")
    if step_index(df_17, "grammer", True) != 0:
        return df_17

    df_last = convert_index(df_17.loc[df_17.errors.apply(len)>0])
    print("Process Done.")

    return df_last

In [55]:
def make_df(df1, df2, s1, s2):
    df1 = random_df(df.loc[nfin_idx], 0.3, s1)
    df2 = random_df(df.loc[fin_idx], 0.8, s2)
    df_samp = pd.concat([df1, df2], axis=0)
    df_samp["curHyp"] = df_samp.KOR.values
    df_samp["errors"] = [[] for _ in range(df_samp.shape[0])]
    df_samp["errLog"] = [dict() for _ in range(df_samp.shape[0])]
    return df_samp

df_samp1 = make_df(df_nfin, df_fin, 111, 123)
df_samp2 = make_df(df_nfin, df_fin, 141, 232)
df_samp3 = make_df(df_nfin, df_fin, 2, 1024)

In [58]:
dindex_dict = {"omission":[], "addition":[], "mis-translation":[], "over-translation":[], "under-translation":[], "grammer": []}
df_res = lets_make_errors(df_samp1, ratios, seed1)
errors_report(df_res , True)

Tokenizing ...
Complete Tokenizing !!!
	1번째 수행 중...완료 !
	2번째 수행 중...완료 !
	3번째 수행 중...완료 !
	4번째 수행 중...완료 !
	5번째 수행 중...완료 !
	6번째 수행 중...완료 !
	7번째 수행 중...완료 !
	8번째 수행 중...완료 !
	9번째 수행 중...완료 !
	10번째 수행 중...완료 !
	11번째 수행 중...완료 !
	12번째 수행 중...완료 !
	13번째 수행 중...완료 !
	14번째 수행 중...완료 !
	15번째 수행 중...완료 !
	16번째 수행 중...완료 !
	17번째 수행 중...완료 !
Process Done.


,omission,addition,mis-translation,over-translation,under-translation,grammer,sum
critical,1.35,0.00,0.00,0.00,0.00,0.00,1.35
major,8.02,14.56,6.21,9.49,5.29,0.00,43.56
minor,0.00,10.59,17.96,0.00,0.00,25.81,54.35
neutral,0.51,0.00,0.23,0.00,0.00,0.00,0.74
sum,9.88,25.14,24.39,9.49,5.29,25.81,100.00


In [61]:
conv_df = convert_index(df_res.loc[df_res.errors.apply(len) > 0])
conv_df.head()

,category,lawName,ENG,KOR,curHyp,errors,errLog
424808,농업,농수산물 품질관리법,A person who intends to obtain an extension of...,품질인증의 유효기간을 연장받으려는 자는 유효기간이 끝나기 전에 해양수산부령으로 정하...,급박의 품질인증의 유효기간을 연장받으려는 자는 유효기간이 끝나기 전에 해양수산대통령...,"[{'severity': 'major', 'start_idx': 0, 'text':...","{'addition': {'start_idx': 0, 'text': '급박의'}, ..."
60286,문화ㆍ공보,저작권법 시행령,A person who intends to obtain authentication ...,법 제56조에 따라 인증을 받으려는 자는 제36조제7항에 따라 고시된 인증기관에 문...,법 제56조에 따라 아편흡식의 인증을 받으려는 자는 제36조제7항에 따라 고시된 인...,"[{'severity': 'major', 'start_idx': 11, 'text'...","{'addition': {'start_idx': 11, 'text': '아편흡식의'..."
266137,형사법,형사소송법,The amended provision of Article 92 (3) shall ...,제92조제3항의 개정규정은 이 법 시행 후 최초로 공소제기 전의 체포·구인·구금이 ...,제92조제3항의 개정규정은 이 법 시행 후 최초로 공소제기 전의 체포·구인·구금이 ...,"[{'severity': 'minor', 'start_idx': 57, 'text'...","{'grammer': {'start_idx': 57, 'text': '적용하다.'}}"
98750,산림,수목원·정원의 조성 및 진흥에 관한 법률,"The term ""arboretum expert or gardening expert...","""수목원 또는 정원 전문가""란 수목유전자원 또는 식물에 대한 지식을 체계적으로 전달...","""수목원 및 정원 전문가""란 수목유전자원 또는 식물에 대한 지식를 체계적으로 전달하...","[{'severity': 'major', 'start_idx': 156, 'text...","{'addition': {'start_idx': 156, 'text': '공사비만의..."
585048,정보통신,통신비밀보호법,The notice referred to in paragraph (5) shall ...,제5항의 규정에 의한 통보서에는 긴급통신제한조치의 목적·대상·범위·기간·집행장소·방...,제5항의 규정에 의한 통보서에는 긴급통신제한조치의 목적·대상·범위·기간·집행장소·방...,"[{'severity': 'minor', 'start_idx': 62, 'text'...","{'addition': {'start_idx': 62, 'text': '하지'}}"


In [ ]:
dindex_dict = {"omission":[], "addition":[], "mis-translation":[], "over-translation":[], "under-translation":[], "grammer": []}
df_res2 = lets_make_errors(df_samp2, ratios, seed2)
errors_report(df_res2, True)

In [ ]:
dindex_dict = {"omission":[], "addition":[], "mis-translation":[], "over-translation":[], "under-translation":[], "grammer": []}
df_res3 = lets_make_errors(df_samp3, ratios, seed3)
errors_report(df_res3, True)

In [ ]:
df_total = pd.concat([df_res1, df_res2, df_res3], axis=0, ignore_index=True)
print(df_total.shape[0])
# print(df_total.drop_duplicates(subset=["errors"]).shape[0])

268569


In [70]:
df_res.head(10)

,category,lawName,ENG,KOR,curHyp,errors,errLog
424808,농업,농수산물 품질관리법,A person who intends to obtain an extension of...,품질인증의 유효기간을 연장받으려는 자는 유효기간이 끝나기 전에 해양수산부령으로 정하...,급박의 품질인증의 유효기간을 연장받으려는 자는 유효기간이 끝나기 전에 해양수산대통령...,"[{'severity': 'major', 'start_idx': 0, 'text':...","{'addition': {'start_idx': 0, 'text': '급박의'}, ..."
60286,문화ㆍ공보,저작권법 시행령,A person who intends to obtain authentication ...,법 제56조에 따라 인증을 받으려는 자는 제36조제7항에 따라 고시된 인증기관에 문...,법 제56조에 따라 아편흡식의 인증을 받으려는 자는 제36조제7항에 따라 고시된 인...,"[{'severity': 'major', 'start_idx': 11, 'text'...","{'addition': {'start_idx': 11, 'text': '아편흡식의'..."
266137,형사법,형사소송법,The amended provision of Article 92 (3) shall ...,제92조제3항의 개정규정은 이 법 시행 후 최초로 공소제기 전의 체포·구인·구금이 ...,제92조제3항의 개정규정은 이 법 시행 후 최초로 공소제기 전의 체포·구인·구금이 ...,"[{'severity': 'minor', 'start_idx': 57, 'text'...","{'grammer': {'start_idx': 57, 'text': '적용하다.'}}"
98750,산림,수목원·정원의 조성 및 진흥에 관한 법률,"The term ""arboretum expert or gardening expert...","""수목원 또는 정원 전문가""란 수목유전자원 또는 식물에 대한 지식을 체계적으로 전달...","""수목원 및 정원 전문가""란 수목유전자원 또는 식물에 대한 지식를 체계적으로 전달하...","[{'severity': 'major', 'start_idx': 156, 'text...","{'addition': {'start_idx': 156, 'text': '공사비만의..."
585048,정보통신,통신비밀보호법,The notice referred to in paragraph (5) shall ...,제5항의 규정에 의한 통보서에는 긴급통신제한조치의 목적·대상·범위·기간·집행장소·방...,제5항의 규정에 의한 통보서에는 긴급통신제한조치의 목적·대상·범위·기간·집행장소·방...,"[{'severity': 'minor', 'start_idx': 62, 'text'...","{'addition': {'start_idx': 62, 'text': '하지'}}"
537611,환경,석면안전관리법 시행규칙,A person who has complied with an order to rec...,영 제10조제2항에 따라 회수 또는 판매금지명령을 이행한 자는 별지 제1호서식의 이...,영 제10조제2항에 따라 회수 및 판매금지명령을 이행한 자는 별지 제1호서식의 이행...,"[{'severity': 'minor', 'start_idx': 17, 'text'...","{'mis-translation': {'start_idx': 17, 'text': ..."
300467,군사,군사법원법,A trial and the pronouncement of a judgment pu...,"즉결심판절차에 따른 심리와 재판의 선고는 공개된 법정에서 하되, 법정은 군사경찰부대...","즉결심판절차에 따른 심리와 재판의 선고는 공개된 법정에서 하되, 법정은 군사경찰부대...","[{'severity': 'minor', 'start_idx': 60, 'text'...","{'grammer': {'start_idx': 60, 'text': '함.'}}"
578642,정보통신,위치정보의 보호 및 이용 등에 관한 법률,"Except as provided in this Act, the provisions...",협회에 관하여 이 법에서 정한 것을 제외하고는 「민법」 중 사단법인에 관한 규정을 ...,협회에 관하여 이 특별법에서 정한 것를 제외하고는 「민법」 중 광산의 사단법인에 관...,"[{'severity': 'major', 'start_idx': 42, 'text'...","{'addition': {'start_idx': 42, 'text': '광산의'},..."
574390,해운,해수욕장의 이용 및 관리에 관한 법률,Where a management authority is provided with ...,관리청은 제2항에 따른 지원이 있는 경우 이에 소요되는 비용 및 시설의 전부 또는 ...,관리청은 제2항에 따른 지원이 있는 경우 이에 소요되는 비용 및 시설의 전부 및 일...,"[{'severity': 'minor', 'start_idx': 43, 'text'...","{'mis-translation': {'start_idx': 43, 'text': ..."
169181,사회복지,한국수화언어법,The State and local governments shall provide ...,"국가와 지방자치단체는 공공행사, 사법·행정 등의 절차, 공공시설 이용, 공영방송, ...","국가과 지방자치단체는 공공행사, 사법·행정 등의 절차, 공공시설 이용, 이용, 공영...","[{'severity': 'minor', 'start_idx': 40, 'text'...","{'addition': {'start_idx': 40, 'text': '이용,'},..."


In [ ]:
df_nd = df_total.drop_duplicates(subset=["curHyp"])
df_nd.shape[0]

265964

In [ ]:
for i in range(1,7):
    print(f"{i}: {round(100*(df_nd.errors.apply(len)==i).sum()/df_nd.shape[0], 2)}")

1: 44.02
2: 37.91
3: 15.01
4: 2.83
5: 0.22
6: 0.0


In [23]:
df_nd = pd.read_csv(os.path.join(gdrive_path, "tot_res1.csv"), encoding="euc-kr", index_col=0)
df_nd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265964 entries, 0 to 268568
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   category  265964 non-null  object
 1   lawName   265964 non-null  object
 2   ENG       265964 non-null  object
 3   KOR       265964 non-null  object
 4   curHyp    265964 non-null  object
 5   errors    265964 non-null  object
 6   errLog    265964 non-null  object
dtypes: object(7)
memory usage: 16.2+ MB


In [42]:
def load_df(df_name, pk1, pk2):
    return_df = pd.read_csv(os.path.join(gdrive_path, df_name), encoding="euc-kr", index_col=0)
    with open(os.path.join(gdrive_path, pk1), "rb") as f:
        return_df["errors"] = pickle.load(f)
    with open(os.path.join(gdrive_path, pk2), "rb") as f:
        return_df["errLog"] = pickle.load(f)
    return return_df

In [44]:
df_res1 = load_df("check_result/df_res1.csv", "check_result/res1_err.pkl", "check_result/res1_errLog.pkl")

In [51]:
errors_report(df_res1)

,omission,addition,mis-translation,over-translation,under-translation,grammer,sum
critical,2135,0,0,0,0,0,2135
major,12671,23028,9833,14998,8346,0,68876
minor,0,16749,28425,0,0,40852,86026
neutral,805,0,360,0,0,0,1165
sum,15611,39777,38618,14998,8346,40852,158202


In [40]:
# df_nd["errors"] = err1
# df_nd["errLog"] =errLog1
# df_nd.head()

In [64]:
temp = check_memorize(df_res)

result = {}
for errType in dindex_dict.keys():
    ttp = temp.loc[temp.errLog.apply(lambda i: errType in i)]
    result[errType] = ttp.loc[ttp["errLog"].apply(lambda x: x[errType]["text"]) != ttp["validation"].apply(lambda y: y[errType])]

In [67]:
result["under-translation"]

,curHyp,errLog,validation
501450,국토교통부장관(법 제82조에 따라 국토교통부장관의 위계만의 권한을 위임·위탁받은 자...,"{'addition': {'start_idx': 28, 'text': '위계만의'}...","{'addition': '위계만의', 'mis-translation': '주', '..."
164388,제11조제1항에 따른 결정에 대하여 최루탄에 의한 이의가 있는 자는 그 결정결과을 ...,"{'addition': {'start_idx': 20, 'text': '최루탄에 의...","{'addition': '최루탄에 의한', 'mis-translation': '',..."
559749,관광단지를 개발하려는 공공기관 등 문화체육관광부령로 정하는 관광단지개발자는 필요하면...,"{'addition': {'start_idx': 58, 'text': '구멍만'},...","{'addition': '업무와', 'mis-translation': '항 단서',..."
136160,유료도로관리권자는 통행료와 부이통행료를 낼 의무가 있는 자가 이를 내지 아니하면 국...,"{'addition': {'start_idx': 116, 'text': '또는'},...","{'addition': '장(', 'under-translation': ' 정하는 ..."
534234,"제1항 단서에 따라 수질오염방지시설(이하 ""방지시설""이라 함)을 설치하지 아니하고 ...","{'under-translation': {'start_idx': 94, 'text'...","{'under-translation': '령으로 정하는 ', 'grammer': '..."
...,...,...,...
356139,사업자는 다음 각 호의 어느 하나에 해당하는 경우에는 제조·수입·판매 또는 제공한 ...,"{'mis-translation': {'start_idx': 185, 'text':...","{'mis-translation': '요한', 'under-translation':..."
453573,산업통상자원부장관은 특별자치시장·시장·군수·구청장로 하여금 대규모점포등법인부인설의 ...,"{'mis-translation': {'start_idx': 87, 'text': ...","{'mis-translation': '수 ', 'under-translation':..."
397349,"금융위원회는 금융지주회사등이 제6조의3, 제6조의4, 제34조, 제36조, 제44조...","{'addition': {'start_idx': 106, 'text': '촉진법만의...","{'addition': '가 제4조', 'mis-translation': '다음 각..."
70945,제1항에 따른 대가는 계약상대자으로부터 대가 지급의 청구를 받은 날부터 명령으로 정...,"{'addition': {'start_idx': 65, 'text': '숙박만의'}...","{'addition': '하며, ', 'under-translation': '숙박만..."


In [ ]:
# df_nd.to_csv(os.path.join(gdrive_path, "tot_res1.csv"), encoding="euc-kr")

# 4.내보내기


In [ ]:
json_dict = make_dict(df_total)

In [ ]:
export_json(json_dict, "data_0828.json")